### Import Data

In [1]:
import pandas as pd
test_df = pd.read_csv('./result/KPM_prompt_based_in_hybrid.csv')
print(test_df.columns)
print(test_df)

Index(['topic', 'stance', 'argument', 'matched_kp(group)',
       'confidence_score(group)', 'matched_kp(1by1)',
       'confidence_score(1by1)'],
      dtype='object')
                                              topic  stance  \
0     Assisted suicide should be a criminal offence       1   
1     Assisted suicide should be a criminal offence       1   
2     Assisted suicide should be a criminal offence       1   
3     Assisted suicide should be a criminal offence       1   
4     Assisted suicide should be a criminal offence       1   
...                                             ...     ...   
7233           The USA is a good country to live in       0   
7234           The USA is a good country to live in       0   
7235           The USA is a good country to live in       0   
7236           The USA is a good country to live in       0   
7237           The USA is a good country to live in       0   

                                               argument  \
0     a cure or

In [4]:
import re

def contains_exception_characters(sentence):
    # Define the pattern for valid characters (English letters, digits, whitespace, and common symbols)
    pattern = r'[^a-zA-Z0-9\s!@#$%^&*()_+\-=\[\]{}|;:\'",.<>/?`~]'

    # Find all exception characters
    exception_chars = re.findall(pattern, sentence)

    # Return True if any exception characters are found, otherwise False
    return bool(exception_chars)

# Example usage
text = "Social networks are the only area of 鈥嬧€媎iscussion where there is no censorship."

result = contains_exception_characters(text)
print(result)  # Output: True


True


In [5]:
for index, row in test_df.iterrows():
    # print(str(index)+": ")
    if contains_exception_characters(test_df.at[index,'argument']):
        print(str(index)+": ")
        # print("old: "+df.at[index,'key_point'])
        # topic = row['topic']
        # argument = row['argument']
        # result = generating(topic, argument)
        # df.at[index, 'key_point'] = result
    # else: print("No")

6621: 
6912: 
7001: 
7036: 


In [2]:
#Initialization, run once.
# test_df['matched_kp'] = ''
# test_df['confidence_score'] = ''
# print(test_df)

In [3]:
# test_df.to_csv('./result/KPM_prompt_based_in_hybrid.csv',index=False)
# print(f"DataFrame has been saved to {'./result/KPM_prompt_based_in_hybrid.csv'}")

In [4]:
prediction_df = pd.read_csv('./result/1by1+predictions_in_group.csv')
print(prediction_df.columns)

Index(['topic', 'stance', 'arguments', 'kep_points',
       'predict_kps(Bert_embedding)', 'predict_kps(avg_embedding)',
       'predict_kps(best_embedding)', 'predict_kps(PCA_embedding)'],
      dtype='object')


### Predict matched KP in group and select best one by ourselves

In [5]:
import openai
import yaml
with open("conf/index.yaml") as f:
    credentials = yaml.safe_load(f)
openai.api_key = credentials['environment_variables']['OPENAI_API_KEY']
model_name = "gpt-4"

In [6]:
def generate_and_eval(topic, argument, kps):
    system_prompt = f"""
        You need to calculate the confidence score of the given argument to each kp of the candidate kps based on the topic
         "{topic}". The confidence score between 0 and 1 means to what extent the argument matches to the kp. 0 represents
         not match, 1 represents match. You need to return a list of lists which contain each kp and the corresponding
         confidence score. The result format is as followed: [[kp1, confidence_score], [kp2, confidence_score],...]
        """
    rep = openai.ChatCompletion.create(
        model=model_name,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": "argument:"+argument+", kps:"+kps}
        ]
    )
    result = rep["choices"][0]["message"]["content"]
    print(result)
    try:
        result_list = ast.literal_eval(result)
        return result_list
    except (SyntaxError, ValueError) as e:
        print(f"Error: {e}. Regenerating string...")
        return generate_and_eval(topic, argument, kps)


In [7]:
import ast

for index,row in test_df.iloc[5500:6000].iterrows():
    topic = row['topic']
    stance = row['stance']
    argument = row['argument']
    filtered_df = prediction_df[(prediction_df['topic']==topic) & (prediction_df['stance']== stance)]
    # print(filtered_df)
    candidate_kps = filtered_df['predict_kps(PCA_embedding)'].values.tolist()
    print(index)
    print(candidate_kps[0])
    input_kps_list = eval(candidate_kps[0])
    # input_kps = ['\"' + sentence.replace('\'', '\"') + '\"' for sentence in input_kps_list]
    input_kps = ['\"' + sentence + '\"' for sentence in input_kps_list]
    # print(str(input_kps))
    # system_prompt = f"""
    #     You need to calculate the confidence score of the given argument to each kp of the candidate kps based on the topic
    #      "{topic}". The confidence score between 0 and 1 means to what extend the argument matches to the kp. 0 represents
    #      not match, 1 represents match. You need to return a list of lists which contain each kp and the corresponding
    #      confidence score. The result format is as followed: [[kp1, confidence_score], [kp2, confidence_score],...]
    #     """
    # rep = openai.ChatCompletion.create(
    #     model=model_name,
    #     messages=[
    #         {"role": "system", "content": system_prompt},
    #         {"role": "user", "content": "argument:"+argument+", kps:"+str(input_kps)}
    #     ]
    # )
    # result = rep["choices"][0]["message"]["content"]
    # result = result.replace("'t", "\\'t") #The problem is the use of a contraction with a single quote within a string that is already enclosed in single quotes 句前有单引号，句子中有单引号会造成判断错误
    # # result = result.replace("'s", "\\'s")
    # result = chatgpt()
    # result_list = eval(result)
    result_list = generate_and_eval(topic,argument, str(input_kps))
    best_score_kp = max(result_list, key=lambda x: x[1])
    best_kp = best_score_kp[0]
    confidence_score = best_score_kp[1]
    best_kp = best_kp.strip('"')
    print("Sentence with the highest score:", best_kp)
    print("Highest score:", confidence_score)
    test_df.at[index,'matched_kp(1by1)'] = best_kp
    test_df.at[index,'confidence_score(1by1)'] = confidence_score

5500
['Flag burning is a form of free speech that democracies should respect.', 'Flag burning, despite its negative perception, should be protected as a freedom, considering it does not directly harm others.', 'The user suggests flag burning as a non-violent form of protest or expression, thus implying it should not be prohibited.', 'Prohibiting flag burning suppresses a form of expression that causes no physical harm or property damage.', 'The freedom of speech should allow all forms of expression, including flag burning, unless they cause harm to others.']
[["Flag burning is a form of free speech that democracies should respect.", 1], ["Flag burning, despite its negative perception, should be protected as a freedom, considering it does not directly harm others.", 0.9], ["The user suggests flag burning as a non-violent form of protest or expression, thus implying it should not be prohibited.", 0.85], ["Prohibiting flag burning suppresses a form of expression that causes no physical ha

In [9]:
test_df.to_csv('./result/KPM_prompt_based_in_hybrid.csv', index=False)
print("Add new data successfully!!!!!")

Add new data successfully!!!!!


### Calculate Tokens

In [14]:
import nltk
from nltk.tokenize import word_tokenize

# Download the Punkt tokenizer for word_tokenize
nltk.download('punkt')

def count_tokens(input_text):
    # Tokenize the input text
    tokens = word_tokenize(input_text)

    # Count the number of tokens
    num_tokens = len(tokens)

    return num_tokens, tokens

# Example usage
input_text = "1.1.1.1.1"
num_tokens, token_list = count_tokens(input_text)

print(f"Input Text: {input_text}")
print(f"Tokens: {token_list}")
print(f"Number of Tokens: {num_tokens}")

Input Text: 1.1.1.1.1
Tokens: ['1.1.1.1.1']
Number of Tokens: 1


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jhluo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
import pandas as pd
token_df = pd.read_csv('./result/KPM_prompt_based_in_hybrid.csv')
prediction_df = pd.read_csv('./result/ArgKP21+predictions(v2).csv')
input_text = ''
output_text = ''
for index, row in token_df.iterrows():
    system_prompt = f"""
        You need to calculate the confidence score of the given argument to each kp of the candidate kps based on the topic
         "{row['topic']}". The confidence score between 0 and 1 means to what extend the argument matches to the kp. 0 represents
         not match, 1 represents match. You need to return a list of lists which contain each kp and the corresponding
         confidence score. The result format is as followed: [[kp1, confidence_score], [kp2, confidence_score],...]
        """
    filtered_df = prediction_df[(prediction_df['topic']==row['topic']) & (prediction_df['stance']== row['stance'])]
    # print(filtered_df)
    candidate_kps = filtered_df['predict_kps'].values.tolist()
    input_kps_list = eval(candidate_kps[0])
    input_kps = ['\"' + sentence + '\"' for sentence in input_kps_list]
    user_content = "argument:"+row['argument']+", kps:"+str(input_kps)
    response_content = (str(row['matched_kp'])+str(row['confidence_score']))*len(input_kps_list)
    input_text += system_prompt+user_content
    output_text += response_content

In [13]:
input_num_tokens, input_token_list = count_tokens(input_text)
output_num_tokens, output_token_list = count_tokens(output_text)
total_cost = input_num_tokens*30/1000000+output_num_tokens*60/1000000

print(f"Number of Input Tokens: {input_num_tokens}")
print(f"Number of Output Tokens: {output_num_tokens}")
print(f"Total cost: {total_cost} $")

Number of Input Tokens: 1742022
Number of Output Tokens: 542351
Total cost: 84.80172 $


### Calculate average confidence score

In [1]:
import pandas as pd
df = pd.read_csv('./result/KPM_prompt_based_in_hybrid.csv')
print(df.columns)

Index(['topic', 'stance', 'argument', 'matched_kp(group)',
       'confidence_score(group)', 'matched_kp(1by1)',
       'confidence_score(1by1)'],
      dtype='object')


In [2]:
score = df['confidence_score(1by1)'].values.tolist()
print(sum(score)/len(score))

0.8608752417794895


### Check matched kp

In [3]:
import pandas as pd
df = pd.read_csv('./result/KPM_prompt_based_in_hybrid.csv')
print(df.columns)
print(df)

Index(['topic', 'stance', 'argument', 'matched_kp(group)',
       'confidence_score(group)', 'matched_kp(1by1)',
       'confidence_score(1by1)'],
      dtype='object')
                                              topic  stance  \
0     Assisted suicide should be a criminal offence       1   
1     Assisted suicide should be a criminal offence       1   
2     Assisted suicide should be a criminal offence       1   
3     Assisted suicide should be a criminal offence       1   
4     Assisted suicide should be a criminal offence       1   
...                                             ...     ...   
7233           The USA is a good country to live in       0   
7234           The USA is a good country to live in       0   
7235           The USA is a good country to live in       0   
7236           The USA is a good country to live in       0   
7237           The USA is a good country to live in       0   

                                               argument  \
0     a cure or

In [4]:
prediction_df = pd.read_csv('./result/1by1+predictions_in_group.csv')
print(prediction_df.columns)

Index(['topic', 'stance', 'arguments', 'kep_points',
       'predict_kps(Bert_embedding)', 'predict_kps(avg_embedding)',
       'predict_kps(best_embedding)', 'predict_kps(PCA_embedding)'],
      dtype='object')


In [6]:
import re
import ast
for index, row in df.iterrows():
    pattern = r'[Kk][Pp]([1-5])'
    match = re.match(pattern, row['matched_kp(group)'])
    if match:
        print(str(index)+": ")
        print(int(match.group(1)))
        kp_id = int(match.group(1))
        topic = row['topic']
        stance = row['stance']
        filtered_df = prediction_df[(prediction_df['topic']==topic) & (prediction_df['stance']== stance)]
        candidate_kps = filtered_df['predict_kps(PCA_embedding)'].values.tolist()
        kps = ast.literal_eval(candidate_kps[0])
        print(kps)
        # print(kps[kp_id-1])
        # df.at[index,'matched_kp(1by1)'] = kps[kp_id-1]